In [1]:
from irep import *
import pandas as pd

In [2]:
# Load up DataFrame and clean cols
df = pd.read_csv('ka.csv')
for col in df.columns:
    if 'unnamed' in col.lower():
        df.drop(col,axis=1,inplace=True)
df.head(10)

,Nationality,Funny_Accent,Waltzing,AU_Anthem,Six,Fish_and_Chips,Favorite,Own_X_Sheep,Can_Fly
0,Aussie,No,No,No,Properly,Normal,Vegemite,Two,No
1,Kiwi,Yes,No,No,Seeks,Feessh_and_Cheeps,Marmite,Three,No
2,Aussie,Yes,No,Some,Seeks,Normal,Marmite,Many,No
3,Kiwi,Yes,Yes,No,Properly,Fush_and_Chups,Vegemite,Many,No
4,Aussie,Yes,No,Yes,Properly,Normal,Marmalade,One,No
5,Aussie,Yes,Yes,No,Seeks,Fush_and_Chups,Butter,Two,No
6,Kiwi,Yes,No,No,Properly,Normal,Marmite,Two,No
7,Aussie,Yes,Yes,No,Properly,Normal,Butter,One,No
8,Kiwi,Yes,No,No,Properly,Normal,Butter,Three,No
9,Aussie,Yes,Yes,No,Seeks,Fush_and_Chups,Butter,Three,No


In [3]:
cond = Cond('Favorite','Marmite')

In [4]:
cond

<Cond object: Favorite=Marmite>

In [5]:
cond.covers(df.head(10))

,Nationality,Funny_Accent,Waltzing,AU_Anthem,Six,Fish_and_Chips,Favorite,Own_X_Sheep,Can_Fly
1,Kiwi,Yes,No,No,Seeks,Feessh_and_Cheeps,Marmite,Three,No
2,Aussie,Yes,No,Some,Seeks,Normal,Marmite,Many,No
6,Kiwi,Yes,No,No,Properly,Normal,Marmite,Two,No


In [6]:
cond.num_covered(df.head(10))

3

In [7]:
rule=Rule()
rule

<Rule object: []>

In [8]:
rule = rule+cond
rule

<Rule object: [Favorite=Marmite]>

In [9]:
rule

<Rule object: [Favorite=Marmite]>

In [10]:
rule = Rule()
rule.grow(df,'Nationality','Kiwi',display=True)
rule

neg 1000
Best Gain: 323.0060802637105
best_successor_rule [Six=Properly]
neg 414
Best Gain: 146.46055744995493
best_successor_rule [Six=Properly^Own_X_Sheep=Three]
neg 41
Best Gain: 10.18579820634384
best_successor_rule [Six=Properly^Own_X_Sheep=Three^Waltzing=No]
neg 10
Best Gain: 2.94340052571314
best_successor_rule [Six=Properly^Own_X_Sheep=Three^Waltzing=No^Funny_Accent=No]


<Rule object: [Six=Properly^Own_X_Sheep=Three^Waltzing=No^Funny_Accent=No]>

In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.33)

In [12]:
irep = IREP(pos_class='Kiwi')
irep.fit(train, 'Nationality', prune=True, display=True)

neg 222
Best Gain: 75.3837079029917
best_successor_rule [Six=Properly]
neg 82
Best Gain: 24.823053286966637
best_successor_rule [Six=Properly^Own_X_Sheep=Three]
neg 11
Best Gain: 3.528212284254705
best_successor_rule [Six=Properly^Own_X_Sheep=Three^Waltzing=No]
neg 2
Best Gain: 0.7215824717726251
best_successor_rule [Six=Properly^Own_X_Sheep=Three^Waltzing=No^Fish_and_Chips=Normal]
Grown: [Six=Properly^Own_X_Sheep=Three^Waltzing=No^Fish_and_Chips=Normal]
Pruned to: [Six=Properly]
Updated ruleset: [[Six=Properly]] 

neg 126
Best Gain: 14.278718143050682
best_successor_rule [Fish_and_Chips=Normal]
neg 35
Best Gain: 7.458790956239011
best_successor_rule [Fish_and_Chips=Normal^Favorite=Marmite]
neg 6
Best Gain: 1.754887502163469
best_successor_rule [Fish_and_Chips=Normal^Favorite=Marmite^Funny_Accent=No]
Grown: [Fish_and_Chips=Normal^Favorite=Marmite^Funny_Accent=No]
Pruned to: [Fish_and_Chips=Normal^Favorite=Marmite^Funny_Accent=No]


In [13]:
irep.ruleset

<Ruleset object: [[Six=Properly]]>

In [14]:
irep.ruleset.covers(df).head(5)

,Nationality,Funny_Accent,Waltzing,AU_Anthem,Six,Fish_and_Chips,Favorite,Own_X_Sheep,Can_Fly
0,Aussie,No,No,No,Properly,Normal,Vegemite,Two,No
3,Kiwi,Yes,Yes,No,Properly,Fush_and_Chups,Vegemite,Many,No
4,Aussie,Yes,No,Yes,Properly,Normal,Marmalade,One,No
6,Kiwi,Yes,No,No,Properly,Normal,Marmite,Two,No
7,Aussie,Yes,Yes,No,Properly,Normal,Butter,One,No


In [15]:
irep.predict(df)[:5]

[True, False, False, True, True]

In [16]:
from sklearn.metrics import accuracy_score, recall_score
precision = irep.score(test.drop('Nationality',axis=1),test['Nationality'],accuracy_score)
recall = irep.score(test.drop('Nationality',axis=1),test['Nationality'],recall_score)
print(f'precision {precision}, recall {recall}')

precision 0.6878787878787879, recall 0.7987987987987988


In [17]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(train.drop('Nationality',axis=1),train['Nationality'])
tree.score(test.drop('Nationality',axis=1),test['Nationality'])

ValueError: could not convert string to float: 'No'